### Download transcripts

In [ ]:
import os
import time
import glob
import json
import random
import re
import asyncio
from playwright.async_api import async_playwright
from playwright.async_api._generated import Page



In [ ]:
# 命名
def sanitize_filename(title):
    """
    Sanitizes a string to be a valid filename by replacing invalid characters.
    """
    return re.sub(r'[<>:"/\\|?*]', '_', title)



In [ ]:
# 解决反爬验证机制
async def wait_for_captcha_and_resume(page: Page):
    """
    Checks for a captcha page and pauses the script for manual resolution.
    """
    print("🤖 Checking for CAPTCHA...")
    try:
        captcha_button = page.get_by_text("按住").first
        await captcha_button.wait_for(state="visible", timeout=5000)
        
        print("\n" + "="*50)
        print("⚠️ CAPTCHA detected! Please manually solve it in the browser.")
        print("💡 Once done, return to the console and press Enter to continue.")
        print("="*50 + "\n")
        
        await asyncio.to_thread(input, "Press Enter to continue...")
        
        await page.wait_for_load_state('domcontentloaded')
        print("✅ CAPTCHA solved, resuming execution.")
        
    except Exception:
        print("🚀 No CAPTCHA detected, continuing.")
        pass



In [ ]:
# 下载单页transcript
async def download_transcript(page: Page, url: str, output_dir: str, title: str):
    """
    Downloads a transcript using an existing Playwright page.
    Returns True if successful, a tuple (False, error_message) otherwise.
    """
    try:
        print(f"Navigating to: {url}")
        
        await page.goto(url, timeout=60000)
        
        await wait_for_captcha_and_resume(page)
        
        await page.wait_for_load_state("networkidle")

        # Use a more specific locator to avoid "strict mode" errors
        transcript_locator = 'div.T2G6W[data-test-id="content-container"]'
        
        error_page_check = page.locator('h1[data-test-id="yikes-page-title"]')
        if await error_page_check.is_visible():
            raise Exception("Page not found on Seeking Alpha.")
        
        await page.wait_for_selector(transcript_locator, timeout=10000)
        
        transcript_div = page.locator(transcript_locator)
        transcript_text = await transcript_div.inner_text()
        
        os.makedirs(output_dir, exist_ok=True)
        
        file_path = os.path.join(output_dir, f"{title}.txt")
        with open(file_path, "w", encoding="utf-8") as f:
            f.write(transcript_text)
        
        print(f"✅ Transcript saved to {file_path}")
        return True

    except Exception as e:
        # Catch any exception and return it as a failure
        return False, str(e)


    

In [ ]:
# 主流程
async def main():
    json_dir = "/Users/mac/Documents/RA/Olivia_Gu/text_analysis/scraper/7/scraper2.0/测试爬取文章/test_json"
    main_output_dir = "/Users/mac/Documents/RA/Olivia_Gu/text_analysis/scraper/7/scraper2.0/测试爬取文章/transcript"
    
    json_files = glob.glob(os.path.join(json_dir, "*.json"))
    print(f"Found {len(json_files)} JSON files to process.")

    async with async_playwright() as p:
        try:
            browser = await p.chromium.connect_over_cdp("http://localhost:9222")
            context = browser.contexts[0]
            page = context.pages[0]
        except Exception as e:
            print(f"❗ Failed to connect to browser instance: {e}")
            print("Please ensure you have run '.../Google Chrome --remote-debugging-port=9222 ...' in the terminal.")
            return

        for file_path in json_files:
            try:
                if os.stat(file_path).st_size == 0:
                    print(f"❗ Skipping empty file: {os.path.basename(file_path)}")
                    continue

                file_name = os.path.basename(file_path)
                ticker = os.path.splitext(file_name)[0].split("_")[-1]

                ticker_output_dir = os.path.join(main_output_dir, ticker)
                os.makedirs(ticker_output_dir, exist_ok=True)
                
                # 定义每个公司的失败日志文件路径
                failed_links_file = os.path.join(ticker_output_dir, "failed.json")
                failed_links = []
                
                downloaded_files = glob.glob(os.path.join(ticker_output_dir, "*.txt"))
                
                print(f"\n--- Processing Ticker: {ticker} ---")
                
                with open(file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                
                results = [(item.get('title'), item.get('link')) for item in data]

                for title, link in results:
                    if not title or not link:
                        continue

                    safe_title = sanitize_filename(title)
                    
                    # 检查是否已下载
                    if any(f"{safe_title}.txt" == os.path.basename(f) for f in downloaded_files):
                        print(f"✅ Transcript for '{title}' already exists. Skipping.")
                        continue
                    
                    clean_url = link.split('#')[0]
                    if 'Earnings' in title:
                        download_result = await download_transcript(page, clean_url, ticker_output_dir, safe_title)
                        
                        # 检查下载结果，如果失败则添加到列表中
                        if isinstance(download_result, tuple):
                            _, error_message = download_result
                            failed_links.append({
                                "title": title,
                                "link": clean_url,
                                "error": error_message
                            })
                        
                        delay = random.uniform(2, 5)
                        print(f"Pausing for {delay:.2f} seconds...")
                        await asyncio.sleep(delay)
                
                # 循环结束后，将该公司的失败链接保存到其专属的 failed.json 文件
                if failed_links:
                    with open(failed_links_file, 'w', encoding='utf-8') as f:
                        json.dump(failed_links, f, indent=4, ensure_ascii=False)
                    print(f"❗ {len(failed_links)} failed links saved to {failed_links_file}")

            except json.JSONDecodeError:
                print(f"❗ Error: {os.path.basename(file_path)} is not a valid JSON file. Skipping.")
                continue
            except Exception as e:
                print(f"❗ An unexpected error occurred while processing {file_path}: {e}")

    print("\n--- All done. ---")

In [ ]:
if __name__ == "__main__":
    try:
        import nest_asyncio
        nest_asyncio.apply()
    except ImportError:
        pass
    
    asyncio.run(main())